# Bibliotecas

In [1]:
import ranking

import pandas as pd
pd.set_option('display.max_colwidth', None)

from math import ceil

# Constantes

In [2]:
COLUNAS = ("descricao_1", "ean_1", "titulo_1", "url_1",	"titulo_cb_1", "loja_1",
           "descricao_2", "ean_2", "titulo_2", "url_2",	"titulo_cb_2", "loja_2",
           "match")
#NUM_AMOSTRAS = 280
DIRETORIO = "Dados/Datasets/"

#lista_nomes = ["celulares", "notebooks", "geladeiras", "fogoes", "tvs"]
lista_nomes = ["celulares"]

In [3]:
df = pd.read_csv(f"Dados/Produtos Processados/pp_{lista_nomes[0]}.csv")
df['ean'].isna().sum()

0

In [4]:
produto = lista_nomes[0]

# Montando os Pares e os Não Pares

Datasets que vou montar:

    1. Hard negative dataset
        Não matches são feitos de acordo com o algoritmo de rankeamento. Pega o não-match mais próximo
    2. Aleatório
        Não matches são feitos aleatóriamente dentro da categoria

In [5]:
def criar_dicionario(descricao_1, ean_1, titulo_1, url_1, titulo_cb_1, loja_1,
                     descricao_2, ean_2, titulo_2, url_2, titulo_cb_2, loja_2,
                     match):
    
    return {'descricao_1' : descricao_1 , 'ean_1' : ean_1, 'titulo_1' : titulo_1,
            'url_1' : url_1, 'titulo_cb_1' : titulo_cb_1, 'loja_1' : loja_1,
            
            'descricao_2' : descricao_2 , 'ean_2' : ean_2, 'titulo_2' : titulo_2,
            'url_2' : url_2 , 'titulo_cb_2' : titulo_cb_2, 'loja_2' : loja_2,
            
            'match' : match
           }

## Pares

In [6]:
def criar_pares(colunas, df_final_dup):

    df_pares = pd.DataFrame(columns = colunas)
    df_pares

    lista_ean = []

    for i in range(len(df_final_dup)):
        
        if df_final_dup.iloc[i]['ean'] not in lista_ean:
            
            lista_ean.append(df_final_dup.iloc[i]['ean'])
            
            indice = df_final_dup.index[df_final_dup['ean'] == df_final_dup.iloc[i]['ean']].tolist()
            
            for j in range(len(indice)-1):
                
                for k in range(j + 1, len(indice)):
                    
                    dicionario = criar_dicionario(descricao_1  = df_final_dup.loc[indice[j]]['descricao'] , ean_1 = df_final_dup.loc[indice[j]]['ean'],
                                                  titulo_1 = df_final_dup.loc[indice[j]]['titulo'], url_1 = df_final_dup.loc[indice[j]]['url'],
                                                  titulo_cb_1 = df_final_dup.loc[indice[j]]['titulo_cb'], loja_1 = df_final_dup.loc[indice[j]]['loja'],
                                                  
                                                  descricao_2 = df_final_dup.loc[indice[k]]['descricao'] , ean_2 = df_final_dup.loc[indice[k]]['ean'],
                                                  titulo_2 = df_final_dup.loc[indice[k]]['titulo'], url_2 = df_final_dup.loc[indice[k]]['url'],
                                                  titulo_cb_2 = df_final_dup.loc[indice[k]]['titulo_cb'], loja_2 = df_final_dup.loc[indice[k]]['loja'],
                                                  
                                                  match = 1
                                                  )
                
                    df_pares = df_pares.append(dicionario, ignore_index = True)

    return df_pares

In [7]:
#df_pares = criar_pares(COLUNAS, df)
#df_pares[df_pares['ean_1'] != df_pares['ean_2']]

In [8]:
#df_pares.to_csv(f"Dados/Pares/pares_{produto}.csv", index = False)

## Não Pares

In [9]:
df_pares = pd.read_csv(f"Dados/Pares/pares_{produto}.csv")

### Aleatório

In [10]:
def criar_nao_pares(df_pares, qtd, colunas = COLUNAS):

    n_matches = df_pares.shape[0]*qtd
    df_nao_pares_aleatorio = pd.DataFrame(columns = colunas)

    for i in range(n_matches):
        created_match = False
        
        while(not created_match):
            df_sorteado = df.sample(n = 2)
            if(df_sorteado.iloc[0]['ean'] != df_sorteado.iloc[1]['ean']):
                
                dic = criar_dicionario(df_sorteado.iloc[0]['descricao'], df_sorteado.iloc[0]['ean'], df_sorteado.iloc[0]['titulo'], df_sorteado.iloc[0]['url'], df_sorteado.iloc[0]['titulo_cb'], df_sorteado.iloc[0]['loja'],
                                    df_sorteado.iloc[1]['descricao'], df_sorteado.iloc[1]['ean'], df_sorteado.iloc[1]['titulo'], df_sorteado.iloc[1]['url'], df_sorteado.iloc[1]['titulo_cb'], df_sorteado.iloc[1]['loja'],
                                    0)
                
                df_nao_pares_aleatorio = df_nao_pares_aleatorio.append(dic, ignore_index = True)
                created_match = True

    return df_nao_pares_aleatorio

#### 5 : 1

In [11]:
#df_nao_pares_aleatorio_5 = criar_nao_pares(df_pares, 5)

In [12]:
#df_nao_pares_aleatorio_5.to_csv(f"Dados/Não Pares/Aléatorio/5 por 1/nao_pares_{produto}.csv", index = False)

#### 1 : 1

In [13]:
#df_nao_pares_aleatorio_1 = criar_nao_pares(df_pares, 1)

In [14]:
#df_nao_pares_aleatorio_1.to_csv(f"Dados/Não Pares/Aléatorio/1 por 1/nao_pares_{produto}.csv", index = False)

#### Conferindo

In [15]:
'''produto = 'celulares'
p = pd.read_csv(f"Dados/Pares/pares_{produto}.csv")
n = pd.read_csv(f"Dados/Não Pares/Aléatorio/1 por 1/nao_pares_{produto}.csv")

print(f"{p.size}\n{n.size}")'''

'produto = \'celulares\'\np = pd.read_csv(f"Dados/Pares/pares_{produto}.csv")\nn = pd.read_csv(f"Dados/Não Pares/Aléatorio/1 por 1/nao_pares_{produto}.csv")\n\nprint(f"{p.size}\n{n.size}")'

## Hard Negative

In [16]:
df_pares = pd.read_csv(f"Dados/Pares/pares_{produto}.csv")

Descobrindo quais EAN se repetem

In [17]:
vc = df['ean'].value_counts()
ean_repetido = vc[vc > 1].index.values

In [18]:
# removendo a pontuação do título
df["titulo_pp"] = df["titulo_cb"].apply(lambda x: ranking.remove_pontuacao(x))

# calculando o tamanho máximo do título
tam_max = max(df.apply(lambda row: len(row["titulo_pp"]), axis = 1))

# calculando o BoW do título
cv, titulo_bow = ranking.formatar_entrada_bow(df['titulo_pp'], mf = tam_max)
#np.unique(titulo_bow)

In [19]:
# calculando a distância entre os vetores
resultado = ranking.calcular_dis_2_vetores(titulo_bow)

c:\Users\llvs2\anaconda3\lib\site-packages\scipy\spatial\distance.py:699: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [20]:
    a

NameError: name 'a' is not defined

In [22]:
# ordenando o resultado
indices, valores = ranking.ordenar_resultado(resultado)

In [ ]:
def criar_nao_pares_hn(df, n_matches, ean_repetido, indices_resultado, colunas = COLUNAS):

    df_nao_pares_hn = pd.DataFrame(columns = colunas)

    if n_matches > len(ean_repetido):
        print("Mais não matches necessários que quantidade de EANS duplicados disponíveis")

    for i in range(n_matches):      

        ean = ean_repetido[i]
        indice_df = df[df['ean'] == ean].head(1).index.values[0] # me da o 1º indice do dataframe onde tem o EAN repetido


        flag_encontrou = False
        j = 0
        while (flag_encontrou == False):
            
            indice_rank = indices_resultado[indice_df][j]

            if df['ean'].loc[indice_rank] != df['ean'].loc[indice_df]:

                dicionario = criar_dicionario(
                                              descricao_1 = df.loc[indice_df]['descricao'] , ean_1 = df.loc[indice_df]['ean'],
                                              titulo_1 =    df.loc[indice_df]['titulo'], url_1 = df.loc[indice_df]['url'],
                                              titulo_cb_1 = df.loc[indice_df]['titulo_cb'], loja_1 = df.loc[indice_df]['loja'],
                                                  
                                              descricao_2 = df.loc[indice_rank]['descricao'] , ean_2 = df.loc[indice_rank]['ean'],
                                              titulo_2 =    df.loc[indice_rank]['titulo'], url_2 = df.loc[indice_rank]['url'],
                                              titulo_cb_2 = df.loc[indice_rank]['titulo_cb'], loja_2 = df.loc[indice_rank]['loja'],
                                                  
                                              match = 0
                                             )
                
                df_nao_pares_hn = df_nao_pares_hn.append(dicionario, ignore_index = True)

                flag_encontrou = True

            j += 1


    return df_nao_pares_hn

In [ ]:
len(ean_repetido)

In [ ]:
df_pares.shape[0]

In [ ]:
t = criar_nao_pares_hn(df = df, n_matches = df_pares.shape[0], ean_repetido = ean_repetido, indices_resultado = indices)

In [ ]:
def criar_nao_pares_hn(df, n_matches, ean_repetido, indices_resultado, colunas = COLUNAS):

    lista_3 = []
    df_nao_pares_hn = pd.DataFrame(columns = colunas)

    limite = ceil(n_matches/len(ean_repetido))

    for i in range(len(ean_repetido)):

        lista_1 = []
        lista_2 = []

        ean = ean_repetido[i]
        indice_df = df[df['ean'] == ean].head(1).index.values[0] # me da o 1º indice do dataframe onde tem o EAN repetido
        lista_1.append(indice_df)

        flag_encontrou = False
        j = 0
        cont = 0
        while (flag_encontrou == False):
            
            indice_rank = indices_resultado[indice_df][j]

            if df['ean'].loc[indice_df] != df['ean'].loc[indice_rank]:


                lista_2.append(indice_rank)

                cont += 1

                if cont > limite:
                    flag_encontrou = True

            j += 1

        lista_3.append([lista_1, lista_2])

    return lista_3

In [ ]:
lista_3 = criar_nao_pares_hn(df = df, n_matches = df_pares.shape[0], ean_repetido = ean_repetido, indices_resultado = indices)

In [ ]:
lista_3[3]

In [ ]:
df[['ean', 'titulo_pp']].iloc[761]

In [ ]:
df[['ean', 'titulo_pp']].iloc[1432]

In [ ]:
df.shape[0]

In [ ]:
len(indices)

In [26]:
df[['ean', 'titulo']].loc[0]

ean                                                                                                                     7892509122580
titulo    Smartphone Samsung Galaxy S22 Ultra 512GB 5G com caneta S pen - Preto, Câmera Tripla 108MP + Selfie 40MP, ram 8GB, Tela 6.8
Name: 0, dtype: object

In [25]:
df[['ean', 'titulo']].loc[1432]

ean                                            7895183061763
titulo    Cômoda 4 Gavetas Flora 4130 Ébano Touch - DEMÓBILE
Name: 1432, dtype: object

In [23]:
indices[0][0:7]

[1432, 1733, 2073, 1348, 1290, 7, 5]

In [24]:
valores[0][0:7]

[1.0, 1.0, 1.0, 1.0, 1.0, 0.9500000000000001, 0.9500000000000001]

In [ ]:
for r in range(len(resultado[0])):

    if resultado[0][r] == 1.0:

        print(r)

# Montando os Datasets

Datasets que vou montar:

    1. Hard negative dataset
        Não matches são feitos de acordo com o algoritmo de rankeamento. Pega o não-match mais próximo
    2. Aleatório
        Não matches são feitos aleatóriamente dentro da categoria

In [ ]:
def salvar_arquivo(df, nome):
    df.to_csv(f"{DIRETORIO}{nome}.csv", index = False)


def embaralhar(df):
    
    df = df.sample(frac = 1)
    
    return df


def criar_dataset(lista_df, nome):
    
    df = pd.concat(lista_df, ignore_index = True)
    
    df = embaralhar(df)
    
    salvar_arquivo(df, nome)

## Hard Negative

### Desbalanceado

In [ ]:
lista_df = []
for nome in lista_nomes:
    
    df = pd.read_csv(f"Dados/Total Pares/total_pares_{nome}.csv")
    lista_df.append(df)

In [ ]:
criar_dataset(lista_df, "hn_desbalanceado")

### Balanceado

In [ ]:
lista_df = []
for nome in lista_nomes:
    
    df_p = pd.read_csv(f"Dados/Pares/pares_{nome}.csv").sample(n = NUM_AMOSTRAS)
    df_np = pd.read_csv(f"Dados/Não Pares/nao_pares_{nome}.csv").sample(n = NUM_AMOSTRAS*5)
    
    lista_df.append(df_p)
    lista_df.append(df_np)

In [ ]:
criar_dataset(lista_df, "hn_balanceado")

## Aleatório

In [ ]:
lista_df = []
for nome in lista_nomes:
    
    df = pd.read_csv(f"Dados/Pares/pares_{nome}.csv")
    lista_df.append(df)
    
df = pd.read_csv(f"Dados/Não Pares/nao_pares_pp_desbalanceado.csv")
lista_df.append(df)

In [ ]:
criar_dataset(lista_df, "sn_desbalanceado")

# Lendo Datasets

In [ ]:
df_hnb = pd.read_csv(f"Dados/Datasets/hn_balanceado.csv")
df_hnd = pd.read_csv(f"Dados/Datasets/hn_desbalanceado.csv")
df_snb = pd.read_csv(f"Dados/Datasets/sn_balanceado.csv")
df_snd = pd.read_csv(f"Dados/Datasets/sn_desbalanceado.csv")

In [ ]:
print(f"Tamanho dos Datasets:\n\n\t\t| Hard\t| Soft\t|\nBalanceado\t| {df_hnb.shape[0]}\t| {df_snb.shape[0]}\t|\nDesbalanceado\t| {df_hnd.shape[0]}\t| {df_snd.shape[0]}\t|")